# Module 3: Locating KITT Using Audio Communication

KITT must be located in its field and then directions must be determined to navigate to the final destination. In the previous modules your colleagues are developing scripts to communicate with KITT. They will add functionality to read the audio signals from the microphones located around the field, and you should use these to locate the car. It is recommended to read Modules 1 and 2 to have a better understanding of how everything should work together.

For the localization, we will use (real-time) recordings of the beacon signal at the various microphones, deconvolve these using a reference signal recording, and determine the relative time delays from the resulting channel estimates. Depending on the distance to each microphone, the signal transmitted by KITT’s beacon arrives a little bit earlier or later, and you can convert that into physical distances. For each pair of microphones, we can compute this time difference of arrival (TDOA), or the physical difference in propagation distance. If you have measurements from enough microphones, then you can calculate the location of KITT in the field.

7 recodings with known locations and a reference recording taken close to the microphone will be provided in this task. This can be used to develop and test your algorithms.

At the end of this Module, you will have developed a script to locate KITT within the field with reason- able accuracy, and you will do so using the data recorded by the microphones located along the field. You will also have tested and verified the accuracy and robustness of your solution.


## Pre-recorded data

These recordings have locations randomly distributed across the field. An example recording with the location are, 

|   x   |   y   |
|-------|-------|
|  64   |   40  |
|  82   |  399  |
|  109  |   76  |
|  143  |  296  |
|  150  |  185  |
|  178  |  439  |
|  232  |  275  |

*Table 1: Sample Data Table*

The x and y axe are defined as follows, where the numbers refer to the microphone index:

```{figure} axisdef.png
---
height/width: 150px
name: mic-figure
---
Microphone Axis definition
```
You can assume these positions for the microphones. Please note the different height of microphone 5.


## Background Knowledge

We will study channel estimation: how do we measure the impulse response from a microphone to a loudspeaker? The specific topics you'll learn in this labday are channel estimation in time-domain (matched filter) and frequency domain. In Labday 4, we will test all this on audio signals.

In [ ]:
import matplotlib.pyplot as plt           # For plotting purposes
import numpy as np                        # For convolution function
import scipy
from scipy import signal                  # For filter function
from scipy.fft import fft, ifft           # For fft and ifft
import math                               # For numerical approximation of pi

## Introduction ##
Suppose we transmit a known signal $x[n]$ over a communication
channel, and measure the result $y[n]$.  The channel acts as a
filter, which we will assume to be linear and time-invariant.
Therefore, the measured signal is a convolution of the transmitted
signal by the channel impulse response $h[n]$, such that
$y[n] = h[n] \ast x[n]$.  Knowing the transmitted signal $x[n]$, can
we recover the impulse response of the communication channel from
$y[n]$?  This is essentially an inversion problem.

Indeed, if we do this in the frequency domain, we have
$
H(\omega) = \frac{Y(\omega)}{X(\omega)}
\,,
$

and we can recover \( h[n] \) from an inverse DTFT. However, there are some complications to do this in the frequency domain. Can we use the DFT instead of the DTFT (which leads to sampling in the frequency domain)?

Are there numerical problems with the above division, what if $X(\omega) = 0$?

Will \( h[n] \) be causal? What about stability?

Alternatively, we can do the deconvolution in the time domain.  This leads to a matrix inversion problem. Also
here, there can be numerical problems.  If the matrix is poorly invertible, then additive noise will be enhanced, and
the resulting channel impulse response estimate will be poor.

We will need channel estimation in the EPO4 project, where each toy
car will transmit audio beacon signals, which are recorded by
microphones at the corners of the field.  From differences in the
estimated channels, we will locate the car.  We also want to be
insensitive to interfering signals: if several beacons are
transmitting audio signals at the same time, we only want to detect
our own transmitting sequence.

Estimation of a propagation channel as studied here is fundamental in many applications. In radar it allows to estimate the distance of objects, in geophysics it represents the reflections at earth layers, in medical ultrasound it allows to form an image of a patient.

To estimate $h[n]$, three alternative algorithms were described:
- Deconvolution in the time domain: Involves matrix inversion. It is computationally complex and
requires lots of memory (easily more than what the available PCs can handle).
- The matched filter: Avoids the matrix inversion. It is equal to computing the cross-correlation of
$y[n]$ with $x[n]$. As cross-correlation is equivalent to a convolution with a reverse signal $x[−n]$, it can be computed efficiently using the FFT. The resulting channel estimate is equal to the true
channel convolved with the autocorrelation of the pulse, $x[n] ∗ x[−n]$. Therefore, its performance
heavily depends on having the correct (i.e., accurate) “reference” or “training” signal to correlate
your measurements with, and the signal should have good autocorrelation properties: $x[n] ∗ x[−n]$
should be close to a delta spike. Large sidelobes will lead to confusion.
- Deconvolution in the frequency domain: Involves the FFT. It computes the same channel as via
deconvolution in the time domain but is much more efficient: convolution in time becomes point-
wise multiplication in frequency, hence for deconvolution in the frequency domain, we only need
a pointwise division, followed by an IFFT to obtain the time-domain channel impulse response.

```{admonition} Note

FFT and IFFT lead to periodicities (the result is cyclic, and samples $x[n]$ at negative time reappear
at the “large n” part of the signal). Since this method is similar to ch1 (but much more efficient),
you can view this method as a matched filter, followed by a correction step that “inverts” the effect
of the autocorrelation of the transmit pulse. However, since we should not divide by zero, you will
also need to implement a “threshold” to set non-invertible or very small frequency coefficients of
$x[n]$ to zero, which otherwise will lead to noise amplification. The performance depends on this
threshold, which has to be chosen heuristically.

```

In each case, a reference signal $x[n]$ is required. We recommend using a recording close to the beacon be-
cause then $x[n]$ includes the loudspeaker and microphone responses as well. For the pre-made recordings,
a high-quality reference is available.
The deconvolution algorithm gives a channel estimate for the beacon path to each microphone. After this,
we detect the first incoming path, which, assuming Line of Sight (LOS), corresponds to the propagation
delay of the car beacon to each microphone. Unfortunately, we do not know the transmit time, so we only
obtain relative propagation delays. If we take the difference of microphone delays, this unknown transmit
time is eliminated, so we can obtain time difference of arrival (TDOA) samples for each microphone pair.
In the next sections, we will use these to locate the car.

## Deconvolution in Frequency Domain ##

Deconvolution in time domain may become complicated if the channel length \( L \) is large.
In that case, $ \mathbf{X} $ becomes too large to invert. At the same time, the Matched Filter may be imprecise and hard to interpret if the autocorrelation sequence \( r[n] \) has large sidelobes. An alternative to the Matched Filter is to do the deconvolution in the frequency domain.

Recall that a convolution in the time domain corresponds to a multiplication in the frequency domain. From $ y[n] = h[n] \ast x[n] $ we can derive $ Y(\omega) = H(\omega) X(\omega) $ and hence we can estimate $ H(\omega) = \frac{Y(\omega)}{X(\omega)} $.
However, this property is valid for the DTFT, whereas in practice we have to use the DFT (or in fact the FFT): we evaluate $\omega $ for only a fixed set of frequencies $ \frac{2\pi}{N} k $, with $ 0 \le k \le N-1 $.

Since we sample in the frequency domain, we can expect some complications: periodicity and aliasing in the time domain! As we will see there, the complications are minimal if we assume the following:
* $ N_y > N_x + L - 1 $, i.e., the training sequence has finite length and the observation is long enough
* The sequences \( x[n] \), \( h[n] \), and \( y[n] \) are zero padded to a common length $ N \ge N_y $.

In this case, let \( X[k] \) be the DFT of \( x[n] \), and similarly for \( H[k] \) and \( Y[k] \), then

$
Y[k] = H[k] X[k] \qquad,  \mbox{for } 0 \le k \le N-1
$

Given $X[k]$ and $Y[k]$, we can compute

$
H[k] = \frac{Y[k]}{X[k]}\,,
\qquad
0 \le k \le N-1
$

and obtain an estimate of the channel in DFT domain.
Next, the inverse Fourier transform is applied to the sequence
$H[k]$ to obtain the channel impulse reponse $\mathbf{h}$.  Together these
steps implement a deconvolution in frequency domain that should be equal to the channel estimate obtained using the matrix inversion technique.  A major advantage is its computational simplicity: we need 3 FFTs (complexity order $3 N \log N$) and a pointwise division (complexity order $N$), rather than a matrix
inversion of the Toeplitz matrix $\mathbf{X}$ (complexity order $N_y\, L^2$).  We also do not need to store large matrices.

This method can be done as follows, 


In [ ]:
#example

x = np.array([1, -0.5])
h = np.array([1, 2, 3, 2, 1])
y = np.convolve(x,h)

Ny = len(y)      # Length of y
Nx = len(x)      # Length of x
L = Ny - Nx + 1    #length of channel (reliable?)

x = np.append(x, [0]* (L-1))     # Make x same length as y

# deconvolution in frequency domain
Y = fft(y)             
X = fft(x)
H = Y/X                 # pointwise division (divide by zero problems?)

# Back transformation
h = np.real(ifft(H))    # ensure the result is real-valued
h = h[0:L]              # optional: truncate to a smaller length L (reliable?)

Also the Matched Filter could be computed in frequency domain. In time
domain, the Matched Filter computes $\hat{h}[n] = y[n] \ast
x[-n]$.  If as before $N_y > N_x + L-1$ and we take care using zero padding that
sequences have equal length $N$, then the equivalent of the Matched Filter in
frequency domain is
$
\hat{H}[k] = Y[k] X^*[k]
$
Also the Matched Filter could be computed in frequency domain. In time
domain, the Matched Filter computes $\hat{h}[n] = y[n] \ast
x[-n]$.  If as before $N_y > N_x + L-1$ and we take care using zero padding that
sequences have equal length $N$, then the equivalent of the Matched Filter in
frequency domain is
$
\hat{H}[k] = Y[k] X^*[k]
$

## Problems with deconvolution in frequency-domain ##

Since a pointwise division is done in frequency domain, it is
immediately clear that the performance will be low for frequencies
where $X[k]$ is small.  Thus, this will only work for training
sequences that resemble either an impulse or "white noise" random
sequences, as these have a wide frequency content. A sinusoid is
the worst training sequence, since its frequency content is almost
zero everywhere. You can't estimate the channel at frequencies where the input signal is zero!

In practice, we should invert $X[k]$ only for frequencies where it
is sufficiently strong, and set the estimates of the remaining
$H[k]$ equal to zero.
Suppose we use a threshold $\epsilon$ and set $\hat{H}[k] =
0$ if $|X[k]| < \epsilon$. Then the channel estimate $\hat{H}[k]$
which we obtain satisfies


$
\hat{H}[k] = H[k] G[k]\,,\qquad \mbox{where }
G[k] = \left\{\begin{array}{ll}
1, & |X[k]| > \epsilon \\
0, & \mbox{elsewhere} \end{array}\right. \quad \quad (6)
$

In time domain, the estimate $\hat{h}[n] = h[n] \ast g[n]$ is the
convolution of the true channel with this "selector function". This
will limit the resolution that can be obtained.

Another problem with frequency-domain equalization is that
there is no guarantee that the computed $\mathbf{h}$ corresponds to an FIR
channel - in general, this will be a vector that has all $N$
entries nonzero.

Moreover, due to frequency-domain sampling, the channel estimate is one period of a periodic sequence. Therefore, in some cases you may observe that the tail (high-$n$) values of $\hat{h}[n]$ are in fact values for small negative $n$. This will in particular be visible if you do delay estimation and the delay is negative.

## Tasks ##
* Implement frequency-domain
equalization as a function `ch3`.  As input parameter, include a threshold
$\epsilon$ on the inversion of $X[k]$. As refinement of the previous discussion, we want $\epsilon$ to be a threshold relative to the peak amplitude in frequency domain: $\epsilon \max_k |X[k]|$. Such a relative threshold makes your function insensitive to a scaling of $X[k]$, which could easily occur if you modify $N$.

* Test the function using the three test signals.

```{hint}
The indices of
the entries of a vector smaller than the relative threshold are found like this:
`ii = np.absolute(X) < epsi*max(np.absolute(X))`.
```

In [ ]:
def ch3(x,y,Lhat,epsi):
    # your code here            # Length of x
    # your code here              # Length of y
    # your code here            # Length of h

    # Force x to be the same length as y
   # your code here

    # Deconvolution in frequency domain
    # your code here

    # Threshold to avoid blow ups of noise during inversion
    # your code here

    #h = # your code here    # ensure the result is real
    #h = # your code here    # optional: truncate to length Lhat (L is not reliable?)
    return h

In [ ]:
# test your ch3 function here:
# Channel
h = np.array([1, 2, 3, 2, 1])
Lhat = 5      # estimate of channel length L, but could be different

# Input length
Nx = 20

# Input: x1, x2, x3
# your code here

## output: y1, y2, y3
# your code here


# Channel estimation via deconvolution: h1, h2, h3
# suitable epsi: try values between 0.001 and 0.05
# your code here

# Print result, should give true channel (which is [1,2,3,2,1])
#print(h1)
#print(h2)
#print(h3)

Suppose we have a training sequence $x[n]$ whose frequency
domain $X[k]$ is bandlimited, e.g., only contains frequencies smaller
than $F_c = 1$ kHz whereas the sample frequency is $F_s = 20$ kHz.

```{figure} XFplot.png
---
height/width: 150px
name: xf-figure
---
Frequency domain plot
```

The effect of the threshold technique on $\hat{h}[n]$ is given by
$(6)$.  What is $G[k]$ in this case?  And $g[n]$?
Make plots.

In [ ]:
Fs = 20     # unit: kHz
Nsamples = 100
#F = #your code here # freq axis 0..10 kHz

#G = #your code here    # Let G = 1 only for 0..1 kHz
#g = #your code here

''' frequency domain
plt.figure(figsize=(10,4))
plt.subplot(121)
plt.plot(#your code here)
plt.xlabel('F [kHz]')
plt.ylabel('Amplitude') '''

# time domain, plot the normalized amplitude (max=1)
'''t = #your code here
plt.subplot(122)
plt.plot(#your code here);
plt.xlabel('t [ms]')
plt.ylabel('Amplitude [normalized]')'''

**Summary**

Channel estimation in frequency domain is generally efficient and accurate.
A threshold is needed to avoid inverting small values in the spectrum of the input signal: we cannot estimate the channel at frequencies which are not in the input signal. The "missing frequencies" determine the accuracy.

In practice, we have training sequences that are bandlimited, and we can estimate channels only in a frequency band. It should not matter too much which band is selected (i.e. the carrier frequency is not important), although the carrier frequency will show up in the channel estimate as a modulation. The bandwidth of the input signal determines the resolution of the channel estimate.

In [1]:
import tkinter as tk
root = tk.Tk()
root.mainloop()